<a href="https://colab.research.google.com/github/harrislam1/covid19_NYC_taxi_trips/blob/EDA_Notebook/NYCTAXI_DASHAPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install jupyter-dash -q

     |████████████████████████████████| 7.3 MB 7.9 MB/s 
     |████████████████████████████████| 23.9 MB 13 kB/s 
     |████████████████████████████████| 357 kB 58.5 MB/s 


In [ ]:
import pandas as pd
import datetime as dt
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  """


In [ ]:
yellow_2020url = "https://nyc-tlc.s3.amazonaws.com/trip+data/yellow_tripdata_2020-"

months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
months_ = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]  # TODO: consider renaming this to month_labels?

total_distance = []
total_fare = []
total_tips = []
total_taxes = []
total_toll = []

daily_trip = pd.DataFrame()

# TODO: elem not very name; consider renaming
for elem in range(len(months)):
  month_data = pd.read_csv(f"{yellow_2020url}{months[elem]}.csv", iterator=True, chunksize=10000)
  month_data = pd.concat(month_data, ignore_index=True)
  month_data.drop(month_data.columns[[0,3,5,6,9,11]], axis=1, inplace=True)
  month_data['pick_up_date'] = pd.to_datetime(month_data.tpep_pickup_datetime)
  month_data['year'] = month_data.pick_up_date.dt.year
  month_data['month'] = month_data.pick_up_date.dt.month
  month_data = month_data.loc[(month_data.month==elem + 1) & (month_data.year==2020)]

  total_distance.append(month_data['trip_distance'].sum())
  total_fare.append(month_data['total_amount'].sum())
  total_tips.append(month_data['tip_amount'].sum())
  total_taxes.append(month_data['mta_tax'].sum())
  total_toll.append(month_data['tolls_amount'].sum())

  
  month_data = month_data.groupby(month_data.pick_up_date.dt.date)['trip_distance'].count().reset_index()
  month_data.rename(columns={'pick_up_date':'Date','trip_distance':'Counts'},inplace=True )
  daily_trip = pd.concat([daily_trip, month_data], ignore_index=True)


# TODO: rename test_df
test_df = pd.DataFrame({'months': months_,
                        'Total Distance Travelled(miles)': total_distance,
                        'Total Fare Collected($ USD)': total_fare,
                        'Total Tips Collected ($ USD)': total_tips,
                        'Total MTA Taxes Collected ($ USD)': total_taxes,
                        'Total Toll Amount Collected ($ USD)': total_toll})

test_df.set_index('months', inplace=True)

# TODO: consider making 2020-03-07 a global variable to make logic more clear
# Build App(Displaying both graphs, include Dropdown menu for second graph)
app = JupyterDash(__name__)
fig1 = px.line(daily_trip, x='Date', y='Counts', title='Daily Trips for Yellow Cabs in 2020').update_layout(
    shapes=[dict(type= 'line', yref= 'paper', y0= 0, y1= 1,xref= 'x', x0= "2020-03-07", x1= "2020-03-07")]).add_annotation(
        x="2020-03-07",text='State of emergency declared')

app.layout = html.Div(children=[
    # All elements from the top of the page
    html.Div([
        html.H1(children="Effect of COVID-19 on NYC's Yellow Cabs in 2020"),
        dcc.Graph(
            id='graph1',
            figure=fig1
        ),  
    ]),
    # New Div for all elements in the new 'row' of the page
    html.Div([
        html.Label(["Options", dcc.Dropdown(id='amount-dropdown',value='Total Distance Travelled(miles)',
                  clearable=False, options=[{'label': c, 'value': c} for c in test_df.columns])]), 
              dcc.Graph(id='graph2')  
    ])
])


# Define callback to update graph
@app.callback(
    Output('graph2','figure'),
    [Input("amount-dropdown", "value")]
)
def update_figure(selected_amount):
  return px.line(x=test_df.index,y=test_df[selected_amount], title=f"Monthly {selected_amount} for Yellow Cabs in 2020",labels={
              "x": "Month","y": f"{selected_amount}"}).update_layout(shapes=
              [dict(type= 'line', yref= 'paper', y0= 0, y1= 1,xref= 'x', x0= 2.1, x1= 2.1)]).add_annotation(
                  x=2.1,text='State of emergency declared')

# Run app and display url result inline in the notebook so you can click url
if __name__=='__main__':
  app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
#daily_trip.to_csv("daily_trip2020.csv", index=False)

In [ ]:
#test_df.to_csv("Monthly_stats2020.csv", index=False)